## EDA

## 🧭 Objetivo del EDA

El propósito de este análisis es comprender la estructura, calidad y comportamientos estadísticos de los **comentarios de Reddit** que servirán como base para entrenar un modelo de **análisis de sentimientos** dentro de un pipeline MLOps.

El EDA busca:

- Identificar problemas de calidad de datos antes del preprocesamiento.
- Evaluar distribuciones, valores extremos y rangos críticos.
- Entender la relación entre variables textuales y el proxy de sentimiento (`score`).
- Detectar riesgos potenciales (desequilibrio, fugas temporales, ruido).
- Derivar decisiones tempranas para el pipeline de ingeniería y modelado.

El detalle completo se encuentra en `01_eda_inicial.ipynb`.

---



In [2]:
import sqlite3
import pandas as pd


db_path = "../db/imedia.sqlite"
conn = sqlite3.connect(db_path)

# Mostrar las tablas disponibles
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tablas disponibles:")
print(tables)

# Leer los datos de cada tabla
authors = pd.read_sql_query("SELECT * FROM authors;", conn)
comments = pd.read_sql_query("SELECT * FROM comments;", conn)
posts = pd.read_sql_query("SELECT * FROM posts;", conn)
subreddits = pd.read_sql_query("SELECT * FROM subreddits;", conn)

# Visualizar primeras filas de cada tabla
print("\n=== Tabla: authors ===")
display(authors.head())

print("\n=== Tabla: comments ===")
display(comments.head())

print("\n=== Tabla: posts ===")
display(posts.head())

print("\n=== Tabla: subreddits ===")
display(subreddits.head())

# Cerrar la conexión
conn.close()


Tablas disponibles:
         name
0  subreddits
1     authors
2       posts
3    comments

=== Tabla: authors ===


,author_name
0,Typical_Wafer_1324
1,krobzaur
2,ITagEveryone
3,ditlevrisdahl
4,geovane_jeff



=== Tabla: comments ===


,comment_id,post_id,author,body,created_utc,parent_id,link_id,score,is_submitter
0,ng4q0es,1nq1588,Present_Tonight1813,I made a program that prompts the user for a s...,1.758810e+09,t3_1nq1588,t3_1nq1588,2,0
1,ng4cg2y,1nq1588,cptsdemon,I made a tool called [PyLiveDev](https://pypi....,1.758805e+09,t3_1nq1588,t3_1nq1588,2,0
2,ng6dq2g,1nq1588,Fr1dge21,As my first project I managed to automate stoc...,1.758827e+09,t3_1nq1588,t3_1nq1588,4,0
3,ng8r3e5,1nq1588,AdventPriest,"Full disclosure, I've leaned heavily on AI to ...",1.758856e+09,t1_ng4aj2a,t3_1nq1588,1,0
4,ng5ys9z,1nq1588,geovane_jeff,My own backup app :D saves me every week!,1.758822e+09,t3_1nq1588,t3_1nq1588,4,0



=== Tabla: posts ===


,post_id,title,selftext,url,permalink,score,num_comments,over_18,created_utc,link_flair_text,is_self,spoiler,locked,thumbnail,subreddit,author
0,1nqnm44,PEP 806 – Mixed sync/async context managers wi...,PEP 806 – Mixed sync/async context managers wi...,https://www.reddit.com/r/Python/comments/1nqnm...,/r/Python/comments/1nqnm44/pep_806_mixed_synca...,107,19,0,1.758847e+09,News,1,0,0,self,PYTHON,kirara0048
1,1nqfyqh,Looking for Feedback and suggestions: Soundmen...,[Soundmentations](https://github.com/saumyarr8...,https://www.reddit.com/r/Python/comments/1nqfy...,/r/Python/comments/1nqfyqh/looking_for_feedbac...,3,0,0,1.758828e+09,Discussion,1,0,0,self,PYTHON,saumyarr8
2,1nq45ep,migrating from django to FastAPI,We've hit the scaling wall with our decade-old...,https://www.reddit.com/r/Python/comments/1nq45...,/r/Python/comments/1nq45ep/migrating_from_djan...,19,53,0,1.758800e+09,Discussion,1,0,0,self,PYTHON,No-Excitement-7974
3,1nq1588,What small Python automation projects turned o...,I’m trying to level up through practice and I’...,https://www.reddit.com/r/Python/comments/1nq15...,/r/Python/comments/1nq1588/what_small_python_a...,141,87,0,1.758788e+09,Discussion,1,0,0,self,PYTHON,MENTX3
4,1nq5x1b,PyCon AU 2025 talks are all up!,This year's PyCon AU talks have all been uploa...,https://www.reddit.com/r/Python/comments/1nq5x...,/r/Python/comments/1nq5x1b/pycon_au_2025_talks...,19,1,0,1.758805e+09,Resource,1,0,0,self,PYTHON,fphhotchips



=== Tabla: subreddits ===


,subreddit,subscribers,description,created_utc,over18
0,PYTHON,1396681,The official Python community for Reddit! Stay...,1.201231e+09,0
1,FUNNY,66831915,Reddit's largest humor depository,1.201243e+09,0
2,PUBLICFREAKOUT,4730514,"A subreddit dedicated to people freaking out, ...",1.381610e+09,0
3,ASKREDDIT,57220281,r/AskReddit is the place to ask and answer tho...,1.201233e+09,0
4,BALDURSGATE3,3229852,"A community all about Baldur's Gate III, the r...",1.559227e+09,0


---

## 📂 0. Contenido y estructura del dataset

El dataset contiene las tablas provenientes de Reddit:

- `comments` (principal)
- `posts`
- `authors`
- `subreddits`

La tabla central **`comments`** incluye las variables relevantes para análisis de sentimiento:

| Variable | Descripción |
|---------|-------------|
| `comment_id` | ID único del comentario |
| `author` | Usuario que comenta |
| `body` | Texto del comentario |
| `score` | Puntuación del comentario (proxy de sentimiento) |
| `is_submitter` | Si el autor del comentario es el creador del post |
| `created_utc` / `created_dt` | Timestamp del comentario |
| `parent_id`, `link_id` | Identificadores de jerarquía |
| `has_author` | Indicador auxiliar (no-nulos en autor) |

Se confirmaron:

- Sin duplicados en `comment_id`.
- Estructura adecuada para procesamiento NLP.

---

In [3]:
# 1. Limpieza mínima: revisar valores faltantes y preparar columnas clave para análisis
comments['has_author'] = comments['author'].notna()

# Conversión de 'created_utc' a datetime para análisis temporal
comments['created_dt'] = pd.to_datetime(comments['created_utc'], unit='s')

# Resumen estructural útil para NLP
display(comments[['author', 'body', 'score', 'is_submitter', 'created_dt']].describe(include='all'))
comments.isna().sum()

# Información estructural
comments.info()

# Conteo de valores nulos
display(comments.isna().sum())

# Proporción de comentarios eliminados ('[deleted]' / '[removed]')
deleted_mask = comments['body'] == '[deleted]'
removed_mask = comments['body'] == '[removed]'

deleted_rate = deleted_mask.mean()
removed_rate = removed_mask.mean()

display({
    "deleted_rate": deleted_rate,
    "removed_rate": removed_rate
})

# Eliminación opcional de comentarios sin contenido útil
comments_clean = comments[~deleted_mask & ~removed_mask].copy()
comments_clean.reset_index(drop=True, inplace=True)


,author,body,score,is_submitter,created_dt
count,4688,4713,4713.000000,4713.000000,4713
unique,3856,4677,NaN,NaN,NaN
top,AndILoveHe,[deleted],NaN,NaN,NaN
freq,15,15,NaN,NaN,NaN
mean,NaN,NaN,56.878421,0.004880,2025-11-13 23:24:29.534478848
min,NaN,NaN,-81.000000,0.000000,2025-09-25 08:38:29
25%,NaN,NaN,1.000000,0.000000,2025-11-11 00:20:19
50%,NaN,NaN,4.000000,0.000000,2025-11-11 06:26:20
75%,NaN,NaN,16.000000,0.000000,2025-11-11 16:21:10
max,NaN,NaN,14829.000000,1.000000,2025-12-02 06:39:20


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4713 entries, 0 to 4712
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   comment_id    4713 non-null   object        
 1   post_id       4713 non-null   object        
 2   author        4688 non-null   object        
 3   body          4713 non-null   object        
 4   created_utc   4713 non-null   float64       
 5   parent_id     4713 non-null   object        
 6   link_id       4713 non-null   object        
 7   score         4713 non-null   int64         
 8   is_submitter  4713 non-null   int64         
 9   has_author    4713 non-null   bool          
 10  created_dt    4713 non-null   datetime64[ns]
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 372.9+ KB


comment_id       0
post_id          0
author          25
body             0
created_utc      0
parent_id        0
link_id          0
score            0
is_submitter     0
has_author       0
created_dt       0
dtype: int64

{'deleted_rate': 0.003182686187141948, 'removed_rate': 0.0014852535539995756}

---

## 🧹 1. Calidad de los datos

### ✔️ Valores faltantes
- `author`: **25 nulos** (~0.5%).
- Resto de columnas sin valores faltantes relevantes.

### ✔️ Comentarios no válidos
- `[deleted]`: 0.318%
- `[removed]`: 0.148%
  
Estos textos no aportan información semántica, por lo que se excluyeron del análisis.

### ✔️ Resultados tras limpieza
```
Retenidos: 4691 comentarios de 4713 (99.53%)
```

La calidad general es buena y apta para modelado supervisado basado en texto.

---

In [4]:
# 2. Distribución del score (sin agrupación automática de Plotly)
import plotly.express as px

# Histogram sin rangos arbitrarios (nbins auto = granular)
fig = px.histogram(
    comments_clean,
    x='score',
    opacity=0.7,
    title="Distribución del score (sin agrupación)",
    marginal="rug"
)
fig.update_layout(bargap=0.05)
fig.show()

# Boxplot de score correctamente construido (caja + bigotes)
fig = px.box(
    comments_clean,
    y="score",
    points=False,               # No scatter; esto limpia visualmente los bigotes
    title="Boxplot del score"
)
fig.show()

# Cuantiles del score para referencia
display(comments_clean['score'].quantile([0.01,0.25,0.50,0.75,0.99]))


0.01      -1.0
0.25       1.0
0.50       4.0
0.75      16.0
0.99    1134.2
Name: score, dtype: float64

---

## 📊 2. Distribución del score (proxy de sentimiento)

El `score` presenta una distribución **extremadamente asimétrica**, con valores muy grandes en la cola derecha.

### Estadísticos clave:
- Mediana = **4**
- Percentil 75 = **16**
- Percentil 99 ≈ **1134**
- Máximo ≈ **14829**

### Implicaciones:
- Es imprescindible **clippear** o transformar el score.
- El uso del score como etiqueta proxy requiere cuidado debido a su alta varianza.
- La mediana es un umbral razonable para construir una primera etiqueta binaria.

📈 Visualizaciones mostradas:
- Histograma sin agrupación artificial.
- Boxplot correctamente formado.


---

In [5]:
# 3. Longitud del texto para análisis de sentimiento
comments_clean['text_len'] = comments_clean['body'].str.len()
comments_clean['word_count'] = comments_clean['body'].str.split().str.len()

# Histograma sin agrupación
fig = px.histogram(
    comments_clean,
    x='text_len',
    nbins=len(comments_clean['text_len'].unique()), # granular al máximo
    opacity=0.7,
    title="Distribución de longitud del texto (sin agrupación)"
)
fig.update_layout(bargap=0.01)
fig.show()

# Boxplot real y limpio de text_len
fig = px.box(
    comments_clean,
    y="text_len",
    points=False,
    title="Boxplot de longitud del texto"
)
fig.show()

# Resumen estadístico
display(comments_clean[['text_len','word_count']].describe())


,text_len,word_count
count,4691.000000,4691.000000
mean,156.683010,27.581965
std,228.139246,39.511984
min,1.000000,1.000000
25%,42.000000,7.000000
50%,93.000000,16.000000
75%,184.000000,33.000000
max,5576.000000,859.000000


---
## ✍️ 3. Análisis de longitud del texto

Se calcularon variables clave para análisis de sentimiento:

- `text_len`: número de caracteres.
- `word_count`: número de palabras.

### Estadísticos:
| Métrica | text_len | word_count |
|---------|----------|------------|
| Media | 156 | 27 |
| Mediana | 93 | 16 |
| Máximo | 5576 | 859 |

### Hallazgos:
- Alta variabilidad en longitudes.
- Fuerte correlación text_len ↔ word_count (≈ 0.99).
- Necesidad de truncation para modelos basados en Transformers.

📈 Visualizaciones incluidas:
- Histograma sin agregación artificial.
- Boxplot limpio de longitud del texto.
- Scatter text_len vs score.
---

In [8]:
# 4. Indicadores de tono emocional
comments_clean['has_exclamation'] = comments_clean['body'].str.contains('!', regex=False)
comments_clean['has_question'] = comments_clean['body'].str.contains('?', regex=False)

# Barras con proporciones
prop_df = comments_clean[['has_exclamation','has_question']].mean()
prop_df = prop_df.reset_index().rename(columns={'index':'feature',0:'proportion'})

fig = px.bar(
    prop_df,
    x='feature',
    y='proportion',
    title="Proporción de signos emocionales en comentarios"
)
fig.show()

# Boxplot score vs exclamación (bien formado)
fig = px.box(
    comments_clean,
    x='has_exclamation',
    y='score',
    points=False,
    title="Score según presencia de exclamación"
)
fig.show()

display(comments_clean[['has_exclamation','has_question']].describe())
print("Proporción de comentarios con exclamación:", comments_clean['has_exclamation'].mean())
print("Proporción de comentarios con pregunta:", comments_clean['has_question'].mean())


,has_exclamation,has_question
count,4691,4691
unique,2,2
top,False,False
freq,4064,3852


Proporción de comentarios con exclamación: 0.13366020038371348
Proporción de comentarios con pregunta: 0.17885312300149223


---
## 😃 4. Indicadores emocionales (features simples)

Se incorporaron señales básicas del texto:

- `has_exclamation`
- `has_question`

### Proporciones:
- Exclamación: **13.36%**
- Interrogación: **17.88%**

### Hallazgos:
- La presencia de `!` o `?` no correlaciona fuertemente con `score`.
- Aun así, pueden aportar información ligera al modelo (features de baja intensidad).

📈 Visualización incluida:
- Barras comparativas de proporciones.
---

In [16]:
# 5. Relación entre score y longitud del comentario
fig = px.scatter(
    comments_clean,
    x="text_len",
    y="score",
    opacity=0.4,
    title="Relación entre longitud del texto y score"
)
fig.show()

# Correlaciones principales
corr = comments_clean[['score','text_len','word_count']].corr()
display(corr)

# Cuartiles de longitud y score con boxplot correctamente formado
comments_clean['len_bucket'] = pd.qcut(
    comments_clean['text_len'],
    q=4,
    labels=['Q1','Q2','Q3','Q4']
)

fig = px.box(
    comments_clean,
    x="len_bucket",
    y="score",
    points=False,
    title="Score por cuartiles de longitud (boxplot correcto)"
)
fig.show()

print("Cuartiles de longitud del texto:")
display(comments_clean['text_len'].quantile([0.25,0.50,0.75]))

print("Cuartiles del score:")
display(comments_clean['score'].quantile([0.25,0.50,0.75]))

print("Correlación entre score y longitud del texto:")
display(comments_clean[['score','text_len']].corr())
display(comments_clean[['score','word_count']].corr())
display(comments_clean[['text_len','word_count']].corr())
display(comments_clean[['score']].describe())


print("Comentarios limpiados:", len(comments_clean), "de", len(comments))
print("Proporción de comentarios retenidos:", len(comments_clean)/len(comments))




,score,text_len,word_count
score,1.000000,-0.004142,-0.002269
text_len,-0.004142,1.000000,0.988565
word_count,-0.002269,0.988565,1.000000


Cuartiles de longitud del texto:


0.25     42.0
0.50     93.0
0.75    184.0
Name: text_len, dtype: float64

Cuartiles del score:


0.25     1.0
0.50     4.0
0.75    16.0
Name: score, dtype: float64

...

,score,text_len
score,1.000000,-0.004142
text_len,-0.004142,1.000000


,score,word_count
score,1.000000,-0.002269
word_count,-0.002269,1.000000


,text_len,word_count
text_len,1.000000,0.988565
word_count,0.988565,1.000000


,score
count,4691.000000
mean,57.125133
std,402.498421
min,-81.000000
25%,1.000000
50%,4.000000
75%,16.000000
max,14829.000000


Comentarios limpiados: 4691 de 4713
Proporción de comentarios retenidos: 0.9953320602588585


---
## 🧩 5. Relación entre score y longitud del comentario

Se evaluó si existe una relación entre la longitud del comentario y su score.

### Resultados:
- Correlación score ↔ text_len ≈ **−0.004**
- Correlación score ↔ word_count ≈ **−0.002**

No existen relaciones lineales significativas.

### Bucketización:
Se analizó score por cuartiles de longitud (`Q1–Q4`), mostrando:

- Distribuciones similares en todos los buckets.
- Longitud NO predice score.

📈 Visualizaciones incluidas:
- Scatter text_len vs score.
- Boxplot de score por cuartiles de longitud.
---

In [26]:
# ===========================
# Chunk EXTRA · Ejemplos de sentimientos (proxy basado en score)
# ===========================

# Calcular la mediana EXACTA del score en comments_clean
median_score = comments_clean["score"].median()

# Crear una columna temporal de sentimiento para mostrar ejemplos
comments_clean["sentiment_bin"] = (comments_clean["score"] >= median_score).astype(int)

print(f"Mediana del score: {median_score}")
print("0 = negativo (score < mediana), 1 = positivo (score >= mediana)")
print("\n")

# Ejemplo negativo (sentiment = 0)
example_neg = (
    comments_clean[comments_clean["sentiment_bin"] == 0]
    .sort_values("score", ascending=True)
    .iloc[0][["score", "body"]]
)

# Ejemplo positivo (sentiment = 1)
example_pos = (
    comments_clean[comments_clean["sentiment_bin"] == 1]
    .sort_values("score", ascending=False)
    .iloc[110][["score", "body"]]
)

print("===== Ejemplo de comentario NEGATIVO (score bajo) =====")
display(example_neg)

print("\n===== Ejemplo de comentario POSITIVO (score alto) =====")
display(example_pos)

# Opcional: borrar columna auxiliar
comments_clean.drop(columns=["sentiment_bin"], inplace=True)


Mediana del score: 4.0
0 = negativo (score < mediana), 1 = positivo (score >= mediana)


===== Ejemplo de comentario NEGATIVO (score bajo) =====


score                                     -81
body     Then what barriers is she breaking? 
Name: 347, dtype: object


===== Ejemplo de comentario POSITIVO (score alto) =====


score                                                404
body     Boil the carcass down for broth for soup later!
Name: 4108, dtype: object

---
## 📌 6. Ejemplos de sentimiento (proxy por score)

Para validar el uso del score como aproximación de sentimiento:

- Se utilizó la mediana del score como umbral para clasificar:
  - 0 → negativo
  - 1 → positivo

Se mostraron ejemplos reales:

- Un comentario con score bajo (negativo).
- Un comentario con score muy alto (positivo).

Estos ejemplos confirmaron que el score puede funcionar como **proxy inicial de sentimiento**, aunque no es perfecto.

---

## ⚠️ 7. Riesgos detectados durante el EDA

### 🔹 Desequilibrio en distribución del score
- La cola derecha es extremadamente larga.
- El modelo podría sesgarse hacia clases de baja puntuación.

### 🔹 Etiquetas proxy imperfectas
- El score no siempre refleja sentimiento auténtico.
- Riesgo de ruido en la supervisión.

### 🔹 Outliers extremos
- Scores mayores a 10k deben ser *clippeados*.

### 🔹 Comentarios largos
- Textos > 5000 caracteres requieren truncación en Transformers.

### 🔹 Riesgo de fuga temporal
- Debe evitarse usar `created_dt` como feature directa sin control temporal.

Estos riesgos motivan un preprocesamiento cuidadoso antes del modelado.

---

## 🛠️ 8. Decisiones tempranas derivadas del EDA

### ✔️ Preprocesamiento necesario
- Remover `[deleted]` y `[removed]`.
- Normalizar texto y limpiar ruido.
- Aplicar clipping → `score_clipped`.
- Crear features adicionales:  
  `text_len_clipped`, `word_count`, `has_exclamation`, `has_question`.

### ✔️ Etiquetado supervisado
- Usar la mediana del score como umbral para obtener `sentiment ∈ {0,1}`.

### ✔️ Ingeniería de características
Variables clave para modelado:

- `clean_text`
- `score_clipped`
- `text_len_clipped`
- `word_count`
- `is_submitter`
- `has_exclamation`
- `has_question`

### ✔️ Lineamiento para MLOps
- Usar embeddings con SentenceTransformer.
- Guardar artefactos reproducibles en `/embeddings` y `/preprocesador`.
- Dividir datos en train/val/test.

Estas decisiones permitirán construir pipelines repetibles, escalables y auditables.
